In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

import multiprocess
# with multiprocess.Pool(processes=8) as pool:
#   for i in pool.imap_unordered(f, a):
#     print(i)

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('19')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [61]:
lines, groups = linesplit("""
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
""".strip())

In [3]:
lines, groups = linesplit(raw_input, verbose=True)

30 lines
1 groups
>>>
Blueprint 1: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 or...
Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 or...
Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 or...
...


In [64]:
ans = []

for bluep_id, line in enumerate(lines):
  a = amap(int, re.findall(r'-?\d+', line))
  id = a[0]
  rob1_1_cost = a[1]
  rob2_1_cost = a[2]
  rob3_1_cost, rob3_2_cost = a[3:5]
  rob4_1_cost, rob4_3_cost = a[5:7]
  
  states = [
    (
     0, 1,
    0, 0,
    0, 0,
    0, 0,
    )
  ]
  # visited = set()
  
  for time in tqdm(range(24)):
    newpareto = set()
    newstates = []
    
    for state in states:
      h1, r1, h2, r2, h3, r3, h4, r4 = state
      
      if (
          h1 >= rob1_1_cost
          and r1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost)
          # and h1 < max(rob1_1_cost, rob2_1_cost, rob3_1_cost, r1 < rob4_1_cost) * 2
      ):
        newstates.append(
          (
            h1 - rob1_1_cost + r1, r1 + 1,
            h2 + r2, r2,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if (
          h1 >= rob2_1_cost
          and r2 < rob3_2_cost
          # and h2 < rob3_2_cost * 2
      ):
        newstates.append(
          (
            h1 - rob2_1_cost + r1, r1,
            h2 + r2, r2 + 1,
            h3 + r3, r3,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob3_1_cost and h2 >= rob3_2_cost and (r3 < rob4_3_cost):
        newstates.append(
          (
            h1 - rob3_1_cost + r1, r1,
            h2 - rob3_2_cost + r2, r2,
            h3 + r3, r3 + 1,
            h4 + r4, r4
          )
        )
      
      if h1 >= rob4_1_cost and h3 >= rob4_3_cost:
        newstates.append(
          (
            h1 - rob4_1_cost + r1, r1,
            h2 + r2, r2,
            h3 - rob4_3_cost + r3, r3,
            h4 + r4, r4 + 1
          )
        )
      
      newstates.append(
        (
          h1 + r1, r1,
          h2 + r2, r2,
          h3 + r3, r3,
          h4 + r4, r4
        )
      )
      
      for i in newstates:
        newpareto.add(i)

    a = -np.array(ndomsort.non_domin_sort(-np.array(list(newpareto)))[0])
    states = [tuple(i) for i in a]
    # newstates = []
    # for i in newpareto:
    #   should_add = True
    #   for j in newpareto:
    #     if i == j:
    #       continue
    #     found_better = True
    #     for ii, jj in zip(i, j):
    #       if jj >= ii:
    #         pass
    #       else:
    #         found_better = False
    #         break
    #     if found_better:
    #       should_add = False
    #       break
    #   if should_add:
    #     newstates.append(i)
      
    # states = newstates
    print(time, len(newpareto), len(states))
  
  ans.append(max([i[-2] for i in states]))
  # break

  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 2 2
3 3 2
4 5 4
5 7 4
6 10 7
7 15 9
8 23 14
9 34 22
10 59 37
11 97 48
12 131 65
13 181 98
14 290 160
15 487 265
16 826 453
17 1436 775
18 2505 1346
19 4546 2358
20 8260 4212
21 15153 8013
22 29486 15716
23 58164 31515


  0%|          | 0/24 [00:00<?, ?it/s]

0 1 1
1 1 1
2 2 2
3 4 3
4 7 4
5 9 7
6 15 9
7 21 9
8 19 9
9 19 7
10 17 11
11 27 15
12 40 23
13 63 36
14 95 51
15 131 68
16 174 88
17 246 134
18 403 208
19 631 314
20 979 506
21 1572 810
22 2488 1285
23 3912 2038


In [65]:
ans

[9, 12]

In [37]:
import scipy.spatial

In [38]:
np.array(list(newpareto))

array([[ 10,   3,  74, ...,   2,   1,   1],
       [ 13,   3,  83, ...,   2,   0,   0],
       [  9,   3,  61, ...,   3,   0,   1],
       ...,
       [ 11,   3,  58, ...,   3,   1,   1],
       [ 12,   3, 116, ...,   0,   0,   0],
       [ 10,   3,  41, ...,   4,   0,   0]])

In [39]:
a = np.array(list(newpareto))
deltas = np.max(a, axis=0) - np.min(a, axis=0)
deltas = deltas[:8]
non_zero_axes = where(deltas > 0)[0]

In [40]:
hull = scipy.spatial.ConvexHull(np.array(states)[:, non_zero_axes])
hull

In [41]:
hull.points.shape, hull.vertices.shape, hull.simplices.shape

((927, 7), (460,), (125444, 7))

In [42]:
len(states)

927

In [43]:
from nds import ndomsort

In [50]:
-np.array(list(newpareto))

array([[ -10,   -3,  -74, ...,   -2,   -1,   -1],
       [ -13,   -3,  -83, ...,   -2,    0,    0],
       [  -9,   -3,  -61, ...,   -3,    0,   -1],
       ...,
       [ -11,   -3,  -58, ...,   -3,   -1,   -1],
       [ -12,   -3, -116, ...,    0,    0,    0],
       [ -10,   -3,  -41, ...,   -4,    0,    0]])

In [55]:
a = -np.array(ndomsort.non_domin_sort(-np.array(list(newpareto)))[0])

In [56]:
a.shape

(927, 8)

927